# TFDV with Titanic Data

## 1. Import Package

In [1]:
import tensorflow as tf
import tensorflow_data_validation as tfdv
print('TF version : ',tf.__version__)
print('TFDV version : ',tfdv.__version__)

TF version :  2.8.0
TFDV version :  1.7.0


## 2. Set Data Path

In [2]:
import os

BASE_DIR = os.path.join('D:\Study\ML\So1s-Study\일섭\week2\titanic', 'kaggle')
DATA_DIR = os.path.join(BASE_DIR, 'data')
OUTPUT_DIR = os.path.join(BASE_DIR, 'output')
TRAIN_DATA = os.path.join(DATA_DIR, 'train.csv')
TEST_DATA = os.path.join(DATA_DIR, 'test.csv')

os.path.isdir(BASE_DIR), os.path.isdir(DATA_DIR), os.path.isdir(OUTPUT_DIR), os.path.isfile(TRAIN_DATA), os.path.isfile(TEST_DATA)

(False, False, False, False, False)

## 3. Train Data Statistics

In [3]:
train_stats = tfdv.generate_statistics_from_csv(data_location=TRAIN_DATA)
tfdv.visualize_statistics(train_stats)

ValueError: No file found in the input data location: D:\Study\ML\So1s-Study\일섭\week2	itanic\kaggle\data\train.csv

## 4. Train Data Schema Check

In [ ]:
schema = tfdv.infer_schema(statistics=train_stats)
tfdv.display_schema(schema=schema)

## 5. Compare Train with Test

In [ ]:
test_stats = tfdv.generate_statistics_from_csv(data_location=TEST_DATA)
tfdv.visualize_statistics(lhs_statistics=test_stats, rhs_statistics=train_stats,
                         lhs_name="TEST_DATA", rhs_name="TRAIN_DATA")

## 6. Check Anomalies (Test Data)

In [ ]:
anomalies = tfdv.validate_statistics(statistics=test_stats, schema=schema)
tfdv.display_anomalies(anomalies)

In [ ]:
options = tfdv.StatsOptions(schema=schema, infer_type_from_schema = True)
test_stats = tfdv.generate_statistics_from_csv(TEST_DATA, stats_options=options)
test_anomalies = tfdv.validate_statistics(test_stats, schema)

schema.default_environment.append('TRAIN')
schema.default_environment.append('TEST')

tfdv.get_feature(schema, 'Survived').not_in_environment.append('TEST')
test_anomalies_with_env = tfdv.validate_statistics(test_stats, schema, environment='TEST')

tfdv.display_anomalies(test_anomalies_with_env)

In [8]:
# Add skew comparator for 'payment_type' feature.
sex = tfdv.get_feature(schema, 'Sex')
sex.skew_comparator.infinity_norm.threshold = 0.01

# # Add drift comparator for 'company' feature.
# sex2=tfdv.get_feature(schema, 'Sex')
# sex2.drift_comparator.infinity_norm.threshold = 0.01

skew_anomalies = tfdv.validate_statistics(train_stats, schema,
                                          serving_statistics=test_stats)

tfdv.display_anomalies(skew_anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'Sex',High Linfty distance between training and serving,"The Linfty distance between training and serving is 0.0112233 (up to six significant digits), above the threshold 0.01. The feature value with maximum difference is: male"
